# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [44]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [45]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [46]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [47]:
# checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

929


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [48]:
# Making a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)
    

#### Create Keyspace

In [49]:
# Creating Keyspace

try: 
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' :1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [50]:
# Connecting to the Keyspace

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query1

### Modeling data - Creating Primary Key

In [51]:
## Song_artist_info is created to find artist, song title and song's length in the music app history that was heard \
## during specific sessionID and itemInSession

# Creating table 
query = "CREATE TABLE IF NOT EXISTS song_artist_info"
query = query + "(artist text, itemInSession int, length double, sessionId int, song text, PRIMARY KEY(artist, itemInSession))"

try:
    session.execute(query)
    print("song_artist_info TABLE CREATED")
except Exception as e:
    print(e)
         

song_artist_info TABLE CREATED


### Do a SELECT to verify the data has been inserted into table properly

In [52]:
query = "SELECT * from song_artist_info"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song)

### Inserting data into the table 'song_artist_info'

In [53]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Inserting info into each assigned column
        query = "INSERT INTO song_artist_info (artist, itemInSession, length, sessionId, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        

## Above columns changed into:         
        session.execute(query, (str(line[0]), int(line[3]), float(line[5]), int(line[8]), str(line[9])))            

#### Validating the data model by a SELECT

In [54]:
## Query 1 shows artist, song title and song's length in the music app history that was heard \
## during sessionId = 338, and itemInSession = 4

query = "SELECT artist, song, length \
         FROM song_artist_info \
         WHERE itemInSession = 4 AND sessionId = 338"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)
    

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## Query 2

### Modeling data - Creating Primary Key and Partition Key

In [55]:
## Song_playlist_session is to search following info: artist, song(sorted by itemInSession) and user(first and last name)\
## palyed in specific by userId and sessionId

# Creating table
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, \
                  firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(query)
    print("song_playlist_session TABLE CREATED")
except Exception as e:
    print(e)
                        

song_playlist_session TABLE CREATED


### Inserting data into the table 'song_playlist_session'

In [56]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        

## Inserting info into each assigned column:        
        query = "INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        
## Above columns changed into:               
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[0]), str(line[9]), str(line[1]), str(line[4])))

### Validating data model by a SELECT 

In [57]:
## Query 2 presents following info: artist name, song(sorted by itemInSession) and user(first and last name)
## for userid = 10, sessionid = 182

query = "SELECT artist, song, firstName, lastName \
         FROM song_playlist_session \
         WHERE userID = 10 AND sessionId = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)
    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3

### Modeling Data - Creating Primary Key, Partition Key

In [58]:
## User_song_history shows every user's full name (first and last) in my music app history 
## who listened to the specific song

## Creating table
query = "CREATE TABLE IF NOT EXISTS user_song_history"
query = query + "(userId int, sessionId int, song text, firstName text, lastName text, artist text, PRIMARY KEY((userId, sessionId), song))"

try:
    session.execute(query)
    
    print("user_song_history TABLE CREATED")
except Exception as e:
    print(e)
    

user_song_history TABLE CREATED


### Inserting data into the table  'user_song_history'

In [59]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Inserting info into each column:   
        query = "INSERT INTO user_song_history (userId, sessionId, song, firstName, lastName, artist)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        
## Above columns changed into:        
        session.execute(query, (int(line[10]), int(line[8]), str(line[9]),  str(line[1]), str(line[4]), str(line[0])))

### Validating table to check if data inserted to table 'user_song_history'

In [64]:
query = "SELECT *\
         FROM user_song_history "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname, row.userid)

Ryan Smith 26
Ryan Smith 26
Ryan Smith 26
Isaac Valdez 3
Isaac Valdez 3
Isaac Valdez 3
Noah Chavez 94
Noah Chavez 94
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Kinsley Young 85
Anabelle Simpson 69
Molly Taylor 35
Katherine Gay 57
Katherine Gay 57
Katherine Gay 57
Katherine Gay 57
Harper Barrett 42
Harper Barrett 42
Harper Barrett 42
Harper Barrett 42
Harper Barrett 42
Harper Barrett 42
Harper Barrett 42
Harper Barrett 42
Harper Barrett 42
Harper Barrett 42
Harp

### Verifiting data model by a SELECT statement 

In [68]:
## Query 3 shows every user's full name (first and last) in my music app history 
## who listened to the song 'All Hands Against His Own'

query = "SELECT firstName, lastName\
         FROM user_song_history \
         WHERE song = 'All Hands Against His Own'" 

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


### Drop the tables before closing out the sessions

In [41]:
## Drop the thee tables before closing out the sessions

In [42]:
query = "DROP TABLE song_artist_info"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE user_song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [43]:

session.shutdown()
cluster.shutdown()